In [1]:
from fastai import *
from fastai.vision.all import *
from fastai.vision.core import *
import torch
from pathlib import Path
import numpy as np
import os
import sys
import shutil
import re
import gc
import pandas as pd
from sklearn.metrics import confusion_matrix
from classifier_common import *

In [2]:
classifier_model = "/data/output/pytorch-extended_v2-20201027_02/classifier-model.pkl"
# data_dir = Path("/data/datasets/Mendeley_Curated_Masked")
# test_dirs = [data_dir/"train"]
data_dir = Path("/data/datasets/extended_v2_Masked")
test_dirs = [data_dir/"evaluate", data_dir/"test"]
regions_csv = data_dir/"regions.csv"
regions_df = pd.read_csv(regions_csv, index_col=0, dtype={"file": str})
regions_df

,x,y,width,height,confidence,class
file,,,,,,
/data/datasets/extended_v2_Masked/train/Pneumonia/SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg,0,59,1692,1355,0.941239,Pneumonia
/data/datasets/extended_v2_Masked/train/Pneumonia/SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg,0,0,1800,1518,0.940427,Pneumonia
/data/datasets/extended_v2_Masked/train/Pneumonia/SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg,0,20,1783,1318,0.909923,Pneumonia
/data/datasets/extended_v2_Masked/train/Pneumonia/SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7.jpeg,0,32,1772,1735,0.974390,Pneumonia
/data/datasets/extended_v2_Masked/train/Pneumonia/SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12.jpeg,0,7,1708,1702,0.977359,Pneumonia
...,...,...,...,...,...,...
/data/datasets/extended_v2_Masked/evaluate/COVID-19/COVID-19-AR-16434409_01.png,356,422,2268,2002,0.934235,COVID-19
/data/datasets/extended_v2_Masked/evaluate/COVID-19/COVID-19-AR-16434444_01.png,368,322,2312,1851,0.908095,COVID-19
/data/datasets/extended_v2_Masked/evaluate/COVID-19/COVID-19-AR-16434444_02.png,343,468,2096,1601,0.929042,COVID-19


In [3]:
cropper = InputCropper(regions_df, min_confidence=0.875)
def get_cropped_image(im_file):
    return cropper.get_cropped_image(str(im_file))

classifier = load_learner(classifier_model)
classifier.freeze()

In [4]:
reverse_class_map = {v: i for i, v in enumerate(classifier.dls.vocab)}

for test_dir in test_dirs:
    gc.collect()
    test_files = get_image_files(test_dir)
    n_files = len(test_files)
    step_size = 256
    p = []
    confidence = []
    
    for i in range(0, n_files, step_size):
        i_end = min(i+step_size, n_files)

        xray_files = test_files[i:i_end]
        dl = classifier.dls.test_dl(xray_files, bs=64)
        preds, _, classes = classifier.get_preds(dl=dl, with_input=False, with_decoded=True)
        conf = preds[np.arange(preds.shape[0]), classes]
        p.extend(classes.tolist())
        confidence.extend(conf.tolist())
        
        print(f"{i_end} / {n_files}")
    
    y = regions_df.loc[[str(f) for f in test_files]]["class"].apply(lambda x: reverse_class_map[x])
#     print(y)
    print(test_dir)
    print(confusion_matrix(y, p, labels=[0,1,2]))

147 / 147
/data/datasets/extended_v2_Masked/evaluate
[[117   1   0]
 [  4  25   0]
 [  0   0   0]]


256 / 1579


512 / 1579


768 / 1579


1024 / 1579


1280 / 1579


1536 / 1579


1579 / 1579
/data/datasets/extended_v2_Masked/test
[[ 94   4   2]
 [  4 863  18]
 [  5  46 543]]
